In [130]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [156]:
startDate = "2023-10-10"
endDate = "2024-06-24"
print(endDate[:4])

2024


In [158]:
# 1. Get full regular season schedule and results
def fetch_schedule(start_date=startDate, end_date=endDate):
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    all_games = []

    while current_date <= end_date:
        date_str = current_date.strftime("%Y-%m-%d")
        url = f"https://api-web.nhle.com/v1/schedule/{date_str}"
        response = requests.get(url)
        
        if response.ok:
            data = response.json()
            for gameDate in data["gameWeek"]:
                tempdate = gameDate["date"]
                for game in gameDate["games"]:
                    if game["gameType"] > 1:
                        try:
                            all_games.append({
                                "game_id": game["id"],
                                "date": tempdate,
                                "home_team": game["homeTeam"]["abbrev"],
                                "home_name": game["homeTeam"]["commonName"]["default"],
                                "away_team": game["awayTeam"]["abbrev"],
                                "away_name": game["awayTeam"]["commonName"]["default"],
                                "home_score": game["homeTeam"]["score"],
                                "away_score": game["awayTeam"]["score"],
                                "game_type": game["gameType"],
                            })
                        except:
                            continue
        current_date += timedelta(days=7)
        # current_date = end_date

    return pd.DataFrame(all_games)

In [160]:
print("Fetching schedule...")
schedule_df = fetch_schedule()
sched_filename = f'nhl_schedule_{startDate[:4]}_{endDate[:4]}.csv'
schedule_df.to_csv(sched_filename, index=False)
schedule_df.head()

Fetching schedule...


,game_id,date,home_team,home_name,away_team,away_name,home_score,away_score,game_type
0,2023020001,2023-10-10,TBL,Lightning,NSH,Predators,5,3,2
1,2023020002,2023-10-10,PIT,Penguins,CHI,Blackhawks,2,4,2
2,2023020003,2023-10-10,VGK,Golden Knights,SEA,Kraken,4,1,2
3,2023020004,2023-10-11,CAR,Hurricanes,OTT,Senators,5,3,2
4,2023020005,2023-10-11,TOR,Maple Leafs,MTL,Canadiens,6,5,2


In [74]:
schedule_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   game_id     1238 non-null   int64 
 1   date        1238 non-null   object
 2   home_team   1238 non-null   object
 3   home_name   1238 non-null   object
 4   away_team   1238 non-null   object
 5   away_name   1238 non-null   object
 6   home_score  1238 non-null   int64 
 7   away_score  1238 non-null   int64 
 8   game_type   1238 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 87.2+ KB


In [ ]:
# 2. Get team stats - We don't know YET what this means etc...
# def fetch_team_stats():
#     team_abbrevs = [
#         "ANA", "ARI", "BOS", "BUF", "CGY", "CAR", "CHI", "COL",
#         "CBJ", "DAL", "DET", "EDM", "FLA", "LAK", "MIN", "MTL",
#         "NSH", "NJD", "NYI", "NYR", "OTT", "PHI", "PIT", "SJS",
#         "SEA", "STL", "TBL", "TOR", "VAN", "VGK", "WSH", "WPG"
#     ]
    
#     stats = []
#     for team in team_abbrevs:
#         url = f"https://api-web.nhle.com/v1/club-stats-season/{team}/20232024"
#         response = requests.get(url)
#         if response.ok:
#             data = response.json()
#             team_stats = data.get("regularSeason", {}).get("total", {})
#             team_stats["team"] = team
#             stats.append(team_stats)
#     return pd.DataFrame(stats)

In [ ]:
# print("Fetching team stats...")
# team_stats_df = fetch_team_stats()
# #team_stats_df.to_csv("nhl_team_stats_2023_2024.csv", index=False)

In [126]:
# 3. Get final standings
def fetch_standings():
    url = "https://api-web.nhle.com/v1/standings/"
    response = requests.get(url)
    teams = []
    if response.ok:
        data = response.json()
        for team in data.get("standings", []):
            teams.append({
                "team_abbr": team.get("teamAbbrev"),
                "games_played": team.get("gamesPlayed"),
                "wins": team.get("wins"),
                "losses": team.get("losses"),
                "ot": team.get("otLosses"),
                "points": team.get("points"),
                "point_pct": team.get("pointPct"),
                "division": team.get("divisionName"),
                "conference": team.get("conferenceName"),
                "rank_division": team.get("divisionSequence"),
                "rank_conference": team.get("conferenceSequence")
            })
    return pd.DataFrame(teams)

In [128]:
print("Fetching standings...")
standings_df = fetch_standings()
standings_filename = f'nhl_standings_{startDate[:4]}_{endDate[:4]}.csv'
standings_df.to_csv(standings_filename, index=False)

Fetching standings...


In [16]:
# 4. Get playoff bracket (matchups + series results)
def fetch_playoffs():
    url = "https://api-web.nhle.com/v1/playoff-bracket"
    response = requests.get(url)
    series_list = []

    if response.ok:
        data = response.json()
        for rnd in data.get("rounds", []):
            for series in rnd.get("series", []):
                series_list.append({
                    "round": rnd.get("number"),
                    "matchup": series.get("matchupAbbrev"),
                    "team1": series["matchupTeams"][0]["abbrev"],
                    "team2": series["matchupTeams"][1]["abbrev"],
                    "team1_wins": series["matchupTeams"][0]["seriesRecord"]["wins"],
                    "team2_wins": series["matchupTeams"][1]["seriesRecord"]["wins"],
                    "status": series.get("seriesStatusShort")
                })
    return pd.DataFrame(series_list)


# Run and save all data
print("Fetching schedule...")
schedule_df = fetch_schedule()
schedule_df.to_csv("nhl_schedule_2023_2024.csv", index=False)

print("Fetching team stats...")
team_stats_df = fetch_team_stats()
team_stats_df.to_csv("nhl_team_stats_2023_2024.csv", index=False)

print("Fetching standings...")
standings_df = fetch_standings()
standings_df.to_csv("nhl_standings_2023_2024.csv", index=False)

print("Fetching playoff bracket...")
playoffs_df = fetch_playoffs()
playoffs_df.to_csv("nhl_playoffs_2024.csv", index=False)

print("✅ All data saved! Ready for modeling and Tableau.")


Fetching schedule...
Fetching team stats...
Fetching standings...
Fetching playoff bracket...
✅ All data saved! Ready for modeling and Tableau.


In [20]:
import requests
import pandas as pd

def fetch_october_schedule():
    all_games = []
    for day in range(1, 32):  # October has 31 days
        date_str = f"2023-10-{day:02d}"
        url = f"https://api-web.nhle.com/v1/schedule/{date_str}"
        response = requests.get(url)
        if response.ok:
            daily_data = response.json()
            for game in daily_data.get("games", []):
                all_games.append({
                    "date": date_str,
                    "home_team": game["homeTeam"]["abbrev"],
                    "away_team": game["awayTeam"]["abbrev"],
                    "home_score": game.get("homeTeam", {}).get("score"),
                    "away_score": game.get("awayTeam", {}).get("score"),
                    "game_state": game.get("gameState")
                })

    df = pd.DataFrame(all_games)
    df.to_csv("nhl_schedule_oct_2023.csv", index=False)
    print(f"✅ Saved {len(df)} games to 'nhl_schedule_oct_2023.csv'.")

fetch_october_schedule()


✅ Saved 0 games to 'nhl_schedule_oct_2023.csv'.


In [98]:
import pprint
import json

# 1. Get full regular season schedule and results
# 2. Get team stats
# 3. Get final standings
# 4. Get playoff bracket (matchups + series results)

# Base URL: https://api-web.nhle.com/
# v1/schedule/now

team_abbrevs = [
    "ANA", "ARI", "BOS", "BUF", "CGY", "CAR", "CHI", "COL",
    "CBJ", "DAL", "DET", "EDM", "FLA", "LAK", "MIN", "MTL",
    "NSH", "NJD", "NYI", "NYR", "OTT", "PHI", "PIT", "SJS",
    "SEA", "STL", "TBL", "TOR", "VAN", "VGK", "WSH", "WPG"
]

df = pd.DataFrame(columns=["team_name", "post_season"])


# url = f"https://api-web.nhle.com/v1/standings/2025-04-04"
# url = "https://api-web.nhle.com/v1/standings-season"
# url = "https://api-web.nhle.com/v1/club-stats/TOR/now"
url = f"https://api-web.nhle.com/v1/club-stats/COL/{startDate[:4]}{endDate[:4]}/2"
response = requests.get(url)
json_data = response.json()

json_data

# for team in team_abbrevs:
#     url = f"https://api-web.nhle.com/v1/club-stats-season/{team}"
#     response = requests.get(url)
#     json_data = response.json()

#     for season in json_data:
#         if season['season'] == int(f'{startDate[:4]}{endDate[:4]}'):
#             if len(season['gameTypes']) > 1:
#                 df.loc[len(df)] = [team, True]
#             else:
#                 df.loc[len(df)] = [team, False]

# df
# pprint.pprint(json_data)
# print(response.text)

{'season': '20242025',
 'gameType': 2,
 'skaters': [{'playerId': 8473446,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20242025/COL/8473446.png',
   'firstName': {'default': 'Erik'},
   'lastName': {'default': 'Johnson'},
   'positionCode': 'D',
   'gamesPlayed': 32,
   'goals': 1,
   'assists': 3,
   'points': 4,
   'plusMinus': 0,
   'penaltyMinutes': 13,
   'powerPlayGoals': 0,
   'shorthandedGoals': 0,
   'gameWinningGoals': 0,
   'overtimeGoals': 0,
   'shots': 27,
   'shootingPctg': 0.037037,
   'avgTimeOnIcePerGame': 843.4688,
   'avgShiftsPerGame': 21.25,
   'faceoffWinPctg': 0.0},
  {'playerId': 8475745,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20242025/COL/8475745.png',
   'firstName': {'default': 'Charlie'},
   'lastName': {'default': 'Coyle'},
   'positionCode': 'C',
   'gamesPlayed': 79,
   'goals': 16,
   'assists': 12,
   'points': 28,
   'plusMinus': -8,
   'penaltyMinutes': 22,
   'powerPlayGoals': 3,
   'shorthandedGoals': 1,
   'gameWinningGoals': 2,
   '